In [94]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import matplotlib.pyplot as plt
import wget
import dask
import os
from tqdm import tqdm
import glob
import os

from numpy import linalg as LA
import math

%matplotlib inline

## Download k-line spot data by every minute from binance

In [2]:
url_template = r"https://data.binance.vision/data/spot/daily/klines/{0}/1h/{0}-1h-{1}.zip"
dir_template = r"data/spot/daily/klines/{0}/1h/{0}-1h-{1}.zip"


# @dask.delayed
def download_one_date(url, path):
    try:
        tmp = wget.download(url, out=path)
        return True
    except:
        # print("{} download failed".format(url))
        return False

def download_all_dates(token_pair: str, dates, path):
    first_meet = False
    for each in dates:
        res = download_one_date(url_template.format(token_pair, str(each.date())), path)
        if first_meet == False and res == True:
            first_meet = True
        if first_meet == True and res == False:
            break
    # promises = [download_one_date(url_template.format(token_pair, str(each.date())), path) for each in dates]
    # alldata=dask.compute(promises) 
    
def get_asset_pairs(x):
    with open("asset_pairs.txt", "r") as f:
        names = f.read()
    names = names.replace("\t", "")
    names = names.replace("\n", "")
    names = names.split("/")
    x_names = list(filter(lambda each: each.endswith(x), names))
    print("x: {} results length: {}".format(x, len(x_names)))
    return x_names
    

In [3]:
@dask.delayed
def main_download(pair, dates):
    path = "data/spot/daily/klines/{0}/1h".format(pair)
    if not os.path.exists(path):
        os.makedirs(path)
    download_all_dates(pair, dates, path)

In [4]:
USDT_pairs = get_asset_pairs("USDT")
BUSD_pairs = get_asset_pairs("BUSD")

x: USDT results length: 394
x: BUSD results length: 350


In [5]:
USDT_pairs = ["BTCUSDT"]
dates = pd.date_range(start="2021-03-01",end="2023-01-15")
promises = [main_download(each, dates) for each in USDT_pairs]
dask.compute(promises)
# for i in tqdm(range(len(USDT_pairs))):
#     main_download(USDT_pairs[i], dates)

([None],)

## Data loading & pre-processing

In [72]:
@dask.delayed
def process_raw(pair, path):
    names = [
        "Open time",
        "Open",
        "High",
        "Low",
        "Close",
        "Volume",
        "Close time",
        "Quote asset volume",
        "Number of trades",
        "Taker buy base asset volume",
        "Taker buy quote asset volume",
        "Ignore",
    ]
    asset_data = pd.read_csv(path, names=names, header=None)
    # btcdata = pd.read_csv(dir_template.format(pair, date), names=names, header=None)
    asset_data["time"] = pd.to_datetime(asset_data["Open time"], unit='ms')
    asset_data[pair] = asset_data["Close"]
    date_indexed = asset_data.set_index("time")
    date_indexed.drop([
        "Open time",
        "Open",
        "High",
        "Low",
        "Close",
        "Volume",
        "Close time",
        "Quote asset volume",
        "Number of trades",
        "Taker buy base asset volume",
        "Taker buy quote asset volume",
        "Ignore",
        ], axis=1, inplace=True)
    # date_indexed.drop('Close time', axis=1, inplace=True)
    
    # date_indexed.drop('time', axis=1, inplace=True)

    # date_indexed["s"] = (date_indexed["isBuyerMaker"].astype(int)-0.5)*(-2)
    # date_indexed["mid"] = date_indexed["price"]
    return date_indexed


def load_one_pair(pair):
    # pair = "BTCUSDT"
    # dates = list(pd.date_range(start="2021-03-01",end="2021-03-06"))
    files = glob.glob("data/spot/daily/klines/{}/1h/*".format(pair))
    # dates = list(pd.date_range(start="2023-01-13",end="2023-01-15"))
    # assert len(files) != 0, f"{pair} is empty, no files found"
    if len(files) == 0:
        print(f"{pair} is empty, no files found")
        return False, None
    tasks = [process_raw(pair, each) for each in files]
    p_data_arr = dask.compute(tasks)
    result = pd.concat(p_data_arr[0])
    return True, result

def merge_assets(pd_arr, col: str):
    assets_close_matrix = pd_arr[0]
    for each in pd_arr[1:]:
        assets_close_matrix = assets_close_matrix.merge(each, how="outer", on=col)
    return assets_close_matrix

In [73]:
def main_load_and_merge_all_assets():
    existing_pairs = os.listdir("data/spot/daily/klines")
    print("Number of pairs: ", len(existing_pairs))
    # assets_pd_arr = [load_one_pair(each) for each in existing_pairs]
    chunk_sz = 10
    subsets = [existing_pairs[i:i + chunk_sz] for i in range(0, len(existing_pairs), chunk_sz)]
    # print(sum([len(each) for each in subsets]))
    for i in tqdm(range(6, len(subsets))):
        chunk = subsets[i]
        tmp_assets_arr = list()
        for each in chunk:
            success, tmp_asset = load_one_pair(each)
            if success:
                tmp_assets_arr.append(tmp_asset)
        # tmp_assets_arr = [load_one_pair(each) for each in chunk]
        tmp_merge_result = merge_assets(tmp_assets_arr, "time")
        tmp_merge_result.to_pickle(f"data/clean/{i}.pkl")
        
    
    # for i in tqdm(range(len(existing_pairs))):
        # assets_pd_arr.append(load_one_pair(existing_pairs[i]))
    # assets_pd_arr = [load_one_pair(each) for each in existing_pairs]
    # return merge_assets(assets_pd_arr, "time")

res = main_load_and_merge_all_assets()

  0%|          | 0/12 [00:00<?, ?it/s]

Number of pairs:  176
GALUSDT is empty, no files found


100%|██████████| 12/12 [16:55<00:00, 84.62s/it]


In [82]:
def merge_all_chunks(arr_id):
    df_arr = [pd.read_pickle(f"data/clean/{i}.pkl") for i in arr_id]
    for each in df_arr:
        print(each)
    res = merge_assets(df_arr, "time")
    return res

In [85]:
res = merge_all_chunks([i for i in range(6)])
len(res.columns)

                     1INCHDOWNUSDT  1INCHUPUSDT  1INCHUSDT  AAVEDOWNUSDT  \
time                                                                       
2021-04-15 07:00:00          10.07         9.87     5.8799      1.327380   
2021-04-15 08:00:00          10.21         9.73     5.8589      1.367000   
2021-04-15 09:00:00           9.92        10.00     5.9137      1.320024   
2021-04-15 10:00:00           9.97         9.93     5.9084      1.301529   
2021-04-15 11:00:00           9.10        10.63     6.1123      1.232953   
...                            ...          ...        ...           ...   
2023-01-15 19:00:00            NaN          NaN        NaN           NaN   
2023-01-15 20:00:00            NaN          NaN        NaN           NaN   
2023-01-15 21:00:00            NaN          NaN        NaN           NaN   
2023-01-15 22:00:00            NaN          NaN        NaN           NaN   
2023-01-15 23:00:00            NaN          NaN        NaN           NaN   

           

60

In [88]:
res
res.to_pickle(f"data/clean/0-5.pkl")
len(res.columns)

60

In [89]:
res1 = merge_all_chunks([i for i in range(6, 12)])
len(res1.columns)
res1.to_pickle(f"data/clean/6-11.pkl")

                     EOSUSDT  ETCUSDT  ETHUSDT  EURUSDT  FILUPUSDT  FIOUSDT  \
time                                                                          
2021-03-01 00:00:00   3.5156  10.4848  1440.76   1.2057      4.274   0.2392   
2021-03-01 01:00:00   3.5163  10.5134  1439.55   1.2041      4.297   0.2226   
2021-03-01 02:00:00   3.5209  10.4562  1440.05   1.2031      4.281   0.2210   
2021-03-01 03:00:00   3.5000  10.3992  1434.06   1.2032      4.294   0.2158   
2021-03-01 04:00:00   3.5076  10.4131  1441.63   1.2029      4.274   0.2175   
...                      ...      ...      ...      ...        ...      ...   
2023-01-15 20:00:00   1.0650  22.8800      NaN   1.0843        NaN      NaN   
2023-01-15 21:00:00   1.0610  22.5200      NaN   1.0837        NaN      NaN   
2023-01-15 22:00:00   1.0640  22.5200      NaN   1.0834        NaN      NaN   
2023-01-15 23:00:00   1.0630  22.4000      NaN   1.0825        NaN      NaN   
2021-06-08 23:00:00      NaN  56.6810  2507.23   1.2

In [91]:
res2 = merge_all_chunks([i for i in range(12, 18)])
len(res2.columns)
res2.to_pickle(f"data/clean/12-17.pkl")

                     PSGUSDT  PYRUSDT  QTUMUSDT  RADUSDT  RAMPUSDT  REEFUSDT  \
time                                                                           
2021-03-01 00:00:00    9.430      NaN     4.998      NaN       NaN  0.029202   
2021-03-01 01:00:00    9.226      NaN     4.971      NaN       NaN  0.029163   
2021-03-01 02:00:00    9.467      NaN     5.000      NaN       NaN  0.029545   
2021-03-01 03:00:00    9.432      NaN     4.931      NaN       NaN  0.029238   
2021-03-01 04:00:00    9.472      NaN     4.955      NaN       NaN  0.029435   
...                      ...      ...       ...      ...       ...       ...   
2023-01-15 19:00:00      NaN     3.85       NaN    1.715       NaN  0.002994   
2023-01-15 20:00:00      NaN     3.85       NaN    1.711       NaN  0.003000   
2023-01-15 21:00:00      NaN     3.81       NaN    1.718       NaN  0.003007   
2023-01-15 22:00:00      NaN     3.80       NaN    1.711       NaN  0.003003   
2023-01-15 23:00:00      NaN     3.76   

In [95]:
pd_whole = merge_assets([res, res1, res2], "time")
print(len(pd_whole.columns))
pd_whole

175


,1INCHDOWNUSDT,1INCHUPUSDT,1INCHUSDT,AAVEDOWNUSDT,AAVEUSDT,AGLDUSDT,AIONUSDT,ALICEUSDT,ALPINEUSDT,AMPUSDT,...,XTZDOWNUSDT,XTZUSDT,YFIDOWNUSDT,YFIIUSDT,YFIUPUSDT,YFIUSDT,YGGUSDT,ZECUSDT,ZENUSDT,ZILUSDT
time,,,,,,,,,,,,,,,,,,,,,
2021-04-15 07:00:00,10.07,9.87,5.8799,1.327380,438.825,NaN,0.45630,12.3072,NaN,NaN,...,0.02294,6.5570,NaN,3044.99,8.890,50826.78,NaN,246.49,96.655,0.20656
2021-04-15 08:00:00,10.21,9.73,5.8589,1.367000,434.066,NaN,0.44530,12.4601,NaN,NaN,...,0.02353,6.5160,NaN,2989.17,8.530,50035.84,NaN,247.08,97.583,0.20509
2021-04-15 09:00:00,9.92,10.00,5.9137,1.320024,439.611,NaN,0.44020,12.5628,NaN,NaN,...,0.02370,6.5169,NaN,3000.41,8.550,50046.70,NaN,248.45,97.487,0.20619
2021-04-15 10:00:00,9.97,9.93,5.9084,1.301529,443.585,NaN,0.43320,12.4980,NaN,NaN,...,0.02487,6.3981,NaN,2954.89,8.301,49476.40,NaN,244.61,95.743,0.20330
2021-04-15 11:00:00,9.10,10.63,6.1123,1.232953,452.599,NaN,0.43930,12.7799,NaN,NaN,...,0.02451,6.4330,NaN,3000.85,8.642,50269.64,NaN,246.76,97.111,0.20582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-15 19:00:00,NaN,NaN,NaN,NaN,80.100,NaN,0.03404,NaN,2.3723,0.00392,...,NaN,1.0220,NaN,1355.00,NaN,7092.00,0.2532,45.30,10.480,0.02575
2023-01-15 20:00:00,NaN,NaN,NaN,NaN,79.900,NaN,0.03497,NaN,2.3822,0.00391,...,NaN,1.0260,NaN,1357.30,NaN,7106.00,0.2592,45.40,10.470,0.02584
2023-01-15 21:00:00,NaN,NaN,NaN,NaN,79.700,NaN,0.03500,NaN,2.3900,0.00389,...,NaN,1.0280,NaN,1357.10,NaN,7068.00,0.2581,45.30,10.480,0.02579


## functions

In [103]:
def eigenvalue_clipping(lambdas,v,lambda_plus):
    N=len(lambdas)
    
    
    # _s stands for _structure below
    sum_lambdas_gt_lambda_plus=np.sum(lambdas[lambdas>lambda_plus])
    
    sel_bulk=lambdas<=lambda_plus                     # these eigenvalues come from the seemingly random bulk
    N_bulk=np.sum(sel_bulk)
    sum_lambda_bulk=np.sum(lambdas[sel_bulk])        
    delta=sum_lambda_bulk/N_bulk                      # delta is their average, so as to conserver the trace of C
    
    lambdas_clean=lambdas
    lambdas_clean[lambdas_clean<=lambda_plus]=delta
    
    
    C_clean=np.zeros((N, N))
    v_m=np.matrix(v)
    
    for i in range(N-1):
        C_clean=C_clean+lambdas_clean[i] * np.dot(v_m[i,].T,v_m[i,]) 
        
    np.fill_diagonal(C_clean,1)
            
    return C_clean    

## Data preprocessing

In [100]:
pd_whole_wo_time = pd_whole.reset_index()
pd_whole_wo_time = pd_whole_wo_time.drop(["time"], axis=1)
pd_whole_wo_time

,1INCHDOWNUSDT,1INCHUPUSDT,1INCHUSDT,AAVEDOWNUSDT,AAVEUSDT,AGLDUSDT,AIONUSDT,ALICEUSDT,ALPINEUSDT,AMPUSDT,...,XTZDOWNUSDT,XTZUSDT,YFIDOWNUSDT,YFIIUSDT,YFIUPUSDT,YFIUSDT,YGGUSDT,ZECUSDT,ZENUSDT,ZILUSDT
0,10.07,9.87,5.8799,1.327380,438.825,NaN,0.45630,12.3072,NaN,NaN,...,0.02294,6.5570,NaN,3044.99,8.890,50826.78,NaN,246.49,96.655,0.20656
1,10.21,9.73,5.8589,1.367000,434.066,NaN,0.44530,12.4601,NaN,NaN,...,0.02353,6.5160,NaN,2989.17,8.530,50035.84,NaN,247.08,97.583,0.20509
2,9.92,10.00,5.9137,1.320024,439.611,NaN,0.44020,12.5628,NaN,NaN,...,0.02370,6.5169,NaN,3000.41,8.550,50046.70,NaN,248.45,97.487,0.20619
3,9.97,9.93,5.9084,1.301529,443.585,NaN,0.43320,12.4980,NaN,NaN,...,0.02487,6.3981,NaN,2954.89,8.301,49476.40,NaN,244.61,95.743,0.20330
4,9.10,10.63,6.1123,1.232953,452.599,NaN,0.43930,12.7799,NaN,NaN,...,0.02451,6.4330,NaN,3000.85,8.642,50269.64,NaN,246.76,97.111,0.20582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16447,NaN,NaN,NaN,NaN,80.100,NaN,0.03404,NaN,2.3723,0.00392,...,NaN,1.0220,NaN,1355.00,NaN,7092.00,0.2532,45.30,10.480,0.02575
16448,NaN,NaN,NaN,NaN,79.900,NaN,0.03497,NaN,2.3822,0.00391,...,NaN,1.0260,NaN,1357.30,NaN,7106.00,0.2592,45.40,10.470,0.02584
16449,NaN,NaN,NaN,NaN,79.700,NaN,0.03500,NaN,2.3900,0.00389,...,NaN,1.0280,NaN,1357.10,NaN,7068.00,0.2581,45.30,10.480,0.02579
16450,NaN,NaN,NaN,NaN,79.900,NaN,0.03493,NaN,2.3882,0.00388,...,NaN,1.0300,NaN,1352.20,NaN,7086.00,0.2573,45.20,10.530,0.02595


In [106]:
t0 = 10000
t1 = 16452
sample = pd_whole_wo_time.iloc[t0:t1].dropna(axis=1)

In [107]:
sample.shape

(5000, 70)